In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometrioid_Cancer"
cohort = "GSE120490"

# Input paths
in_trait_dir = "../../input/GEO/Endometrioid_Cancer"
in_cohort_dir = "../../input/GEO/Endometrioid_Cancer/GSE120490"

# Output paths
out_data_file = "../../output/preprocess/Endometrioid_Cancer/GSE120490.csv"
out_gene_data_file = "../../output/preprocess/Endometrioid_Cancer/gene_data/GSE120490.csv"
out_clinical_data_file = "../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE120490.csv"
json_path = "../../output/preprocess/Endometrioid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Improving Risk Assessment for Metastatic Disease in Endometrioid Endometrial Cancer Patients Using Molecular and Clinical Features: an NRG Oncology / Gynecologic Oncology Group Study"
!Series_summary	"Accurate methods to predict nodal and distant metastasis are needed in endometrioid endometrial cancer (EEC) patients to advance personalized care and reduce both overtreatment and undertreatment.  A transcript-based classifier for predicting risk of nodal and distant metastasis in EEC patients was developed, and shown to outperform a panel of clinical and molecular features"
!Series_summary	"We used microarrays to detail the gene expression in EEC patients and identified a classifer to predict nodal and distant metastasis"
!Series_overall_design	"Frozen primary endometioid endometrial cancer tissues acquired at the time of primary surgical staging undrwent transcriptomic analysis using the Affymetrix U133 Plus 2.0 microarray platform."
!Series_overa

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on background information, this dataset contains transcriptomic data from Affymetrix microarrays
# which includes gene expression
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Key 0: 'matastasis: No', 'matastasis: Yes' - This relates to cancer metastasis status
# For Endometrioid_Cancer, metastasis status is a relevant outcome variable
trait_row = 0

# No information about age in the sample characteristics
age_row = None

# No information about gender (though this is endometrial cancer, likely all female patients)
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        status = value.split(':', 1)[1].strip().lower()
        if status == 'yes':
            return 1
        elif status == 'no':
            return 0
    return None

# Age conversion function (not used)
def convert_age(value):
    return None

# Gender conversion function (not used)
def convert_gender(value):
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial validation and filtering
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We need to check if the clinical_data.csv file exists and handle accordingly
import os

if trait_row is not None:
    clinical_data_path = f"{in_cohort_dir}/clinical_data.csv"
    
    if os.path.exists(clinical_data_path):
        # File exists, proceed with normal processing
        clinical_data = pd.read_csv(clinical_data_path, index_col=0)
        
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the data
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save the processed clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
    else:
        print(f"Clinical data file not found at {clinical_data_path}")
        print("Cannot process clinical data without the source file.")


Clinical data file not found at ../../input/GEO/Endometrioid_Cancer/GSE120490/clinical_data.csv
Cannot process clinical data without the source file.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 64
Header line: "ID_REF"	"GSM3401488"	"GSM3401489"	"GSM3401490"	"GSM3401491"	"GSM3401492"	"GSM3401493"	"GSM3401494"	"GSM3401495"	"GSM3401496"	"GSM3401497"	"GSM3401498"	"GSM3401499"	"GSM3401500"	"GSM3401501"	"GSM3401502"	"GSM3401503"	"GSM3401504"	"GSM3401505"	"GSM3401506"	"GSM3401507"	"GSM3401508"	"GSM3401509"	"GSM3401510"	"GSM3401511"	"GSM3401512"	"GSM3401513"	"GSM3401514"	"GSM3401515"	"GSM3401516"	"GSM3401517"	"GSM3401518"	"GSM3401519"	"GSM3401520"	"GSM3401521"	"GSM3401522"	"GSM3401523"	"GSM3401524"	"GSM3401525"	"GSM3401526"	"GSM3401527"	"GSM3401528"	"GSM3401529"	"GSM3401530"	"GSM3401531"	"GSM3401532"	"GSM3401533"	"GSM3401534"	"GSM3401535"	"GSM3401536"	"GSM3401537"	"GSM3401538"	"GSM3401539"	"GSM3401540"	"GSM3401541"	"GSM3401542"	"GSM3401543"	"GSM3401544"	"GSM3401545"	"GSM3401546"	"GSM3401547"	"GSM3401548"	"GSM3401549"	"GSM3401550"	"GSM3401551"	"GSM3401552"	"GSM3401553"	"GSM3401554"	"GSM3401555"	"GSM3401556"	"GSM3401557"	"GSM3401558"	"GSM3401559"	"GSM3401560"	

Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers shown in the output
# The identifiers like "1007_s_at", "1053_at", etc. are probe IDs from Affymetrix microarrays
# These are not human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE120490
Line 6: !Series_title = Improving Risk Assessment for Metastatic Disease in Endometrioid Endometrial Cancer Patients Using Molecular and Clinical Features: an NRG Oncology / Gynecologic Oncology Group Study
Line 7: !Series_geo_accession = GSE120490
Line 8: !Series_status = Public on Dec 30 2022
Line 9: !Series_submission_date = Sep 26 2018
Line 10: !Series_last_update_date = Dec 31 2022
Line 11: !Series_pubmed_id = 36077609
Line 12: !Series_summary = Accurate methods to predict nodal and distant metastasis are needed in endometrioid endometrial cancer (EEC) patients to advance personalized care and reduce both overtreatment and undertreatment.  A transcript-based classifier for predicting risk of 


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DE

### Step 6: Gene Identifier Mapping

In [7]:
# Identify the column keys for gene identifiers and gene symbols
# Based on the annotation dataframe preview, 'ID' contains probe IDs like in gene expression data,
# and 'Gene Symbol' contains the gene symbols we need

# 1. Extract the gene mapping dataframe
prob_col = 'ID'
gene_col = 'Gene Symbol'
mapping_data = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 2. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)

# 3. Preview the converted gene expression data
print("\nAfter mapping to gene symbols, first 5 genes:")
print(gene_data.head())
print(f"\nShape of gene expression data: {gene_data.shape}")



After mapping to gene symbols, first 5 genes:
          GSM3401488  GSM3401489  GSM3401490  GSM3401491  GSM3401492  \
Gene                                                                   
A1BG        3.283200    4.132405    3.481172    3.323572    3.495063   
A1BG-AS1    4.267424    4.628276    4.042274    4.158514    4.443112   
A1CF        6.045227    6.201178    6.234571    6.156238    5.708011   
A2M        12.383235   12.459452   11.975975   11.046249   12.578974   
A2M-AS1     4.567768    3.955607    4.611827    3.969364    3.972930   

          GSM3401493  GSM3401494  GSM3401495  GSM3401496  GSM3401497  ...  \
Gene                                                                  ...   
A1BG        4.160835    3.313960    5.923445    4.478525    3.389833  ...   
A1BG-AS1    5.034340    4.219107    4.812417    4.510440    4.372276  ...   
A1CF        6.374200    6.086950    6.151695    5.717508    6.083088  ...   
A2M        13.598674   12.810138   11.553073   11.842456   11.8

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (19845, 145)
First few genes with their expression values after normalization:
          GSM3401488  GSM3401489  GSM3401490  GSM3401491  GSM3401492  \
Gene                                                                   
A1BG        3.283200    4.132405    3.481172    3.323572    3.495063   
A1BG-AS1    4.267424    4.628276    4.042274    4.158514    4.443112   
A1CF        6.045227    6.201178    6.234571    6.156238    5.708011   
A2M        12.383235   12.459452   11.975975   11.046249   12.578974   
A2M-AS1     4.567768    3.955607    4.611827    3.969364    3.972930   

          GSM3401493  GSM3401494  GSM3401495  GSM3401496  GSM3401497  ...  \
Gene                                                                  ...   
A1BG        4.160835    3.313960    5.923445    4.478525    3.389833  ...   
A1BG-AS1    5.034340    4.219107    4.812417    4.510440    4.372276  ...   
A1CF        6.374200    6.086950    6.151695    5.717508    6.083088  ...   
A2M

Normalized gene data saved to ../../output/preprocess/Endometrioid_Cancer/gene_data/GSE120490.csv


Raw clinical data shape: (3, 146)
Clinical features:
                     GSM3401488  GSM3401489  GSM3401490  GSM3401491  \
Endometrioid_Cancer         0.0         0.0         0.0         0.0   

                     GSM3401492  GSM3401493  GSM3401494  GSM3401495  \
Endometrioid_Cancer         0.0         0.0         0.0         0.0   

                     GSM3401496  GSM3401497  ...  GSM3401623  GSM3401624  \
Endometrioid_Cancer         0.0         0.0  ...         0.0         0.0   

                     GSM3401625  GSM3401626  GSM3401627  GSM3401628  \
Endometrioid_Cancer         0.0         0.0         0.0         0.0   

                     GSM3401629  GSM3401630  GSM3401631  GSM3401632  
Endometrioid_Cancer         1.0         1.0         1.0         1.0  

[1 rows x 145 columns]
Clinical features saved to ../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE120490.csv
Linked data shape: (145, 19846)
Linked data preview (first 5 rows, first 5 columns):
            Endom

Data shape after handling missing values: (145, 19846)
For the feature 'Endometrioid_Cancer', the least common label is '1.0' with 45 occurrences. This represents 31.03% of the dataset.
The distribution of the feature 'Endometrioid_Cancer' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Endometrioid_Cancer/cohort_info.json


Linked data saved to ../../output/preprocess/Endometrioid_Cancer/GSE120490.csv
